In [6]:
import scanpy as sc
import numpy as np
import scipy.sparse as sp
from sklearn.neighbors import NearestNeighbors
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv
from torch_geometric.data import Data
import torch.optim as optim
from sklearn.cluster import KMeans

In [7]:
#file = r"C:\Users\ouyan\Downloads\BrainAgingSpatialAtlas_Imputed.h5ad"
#adata_full = sc.read_h5ad(file, backed='r')

#print(adata_full)
#print("obs columns:", adata_full.obs.columns)
#print("var columns:", adata_full.var.columns)
#print("obsm keys:", adata_full.obsm_keys())
#print("layers:", adata_full.layers.keys())

In [8]:
#print("Available slice IDs:", adata_full.obs['slice'].unique())

In [9]:
# 截取子集并复制到内存
#TEST_SLICE_ID = '1'
#print(f"2. Creating subset for slice ID '{TEST_SLICE_ID}'...")
#adata = adata_full[adata_full.obs['slice'] == TEST_SLICE_ID, :].to_memory() 

# 释放 full data 的引用
#del adata_full
#print(f"✅ Subset created. New shape: {adata.shape}")

In [10]:
# --- A. 数据加载与子集创建 (最终修正版) ---

file = r"C:\Users\ouyan\Downloads\BrainAgingSpatialAtlas_Imputed.h5ad"
TEST_SLICE_ID = '1'

print(f"1. Loading data in backed mode to conserve memory...")
adata_full = sc.read_h5ad(file, backed='r') 

# 2. 截取子集并显式加载到内存
print(f"2. Creating subset for slice ID '{TEST_SLICE_ID}' and forcing memory conversion...")

# 步骤 2a: 切片
adata_subset_view = adata_full[adata_full.obs['slice'] == TEST_SLICE_ID, :]

# 步骤 2b: 将切片视图转换为一个全新的 AnnData 对象 (通常这就会解决问题)
adata = adata_subset_view.to_memory()

# 步骤 2c: 🚨 核心修正：确保 X 是标准的 scipy sparse matrix (CSR)
# 这一步将 _CSRDataset 明确转换为标准的 CSRMatrix，以通过 scanpy 的类型检查
if not isinstance(adata.X, sp.csr_matrix):
    adata.X = adata.X.tocsr()
    print("   -> adata.X successfully converted to scipy.sparse.csr_matrix.")


# 释放 full data 的引用
del adata_full
print(f"✅ Subset created. New shape: {adata.shape}")

1. Loading data in backed mode to conserve memory...
2. Creating subset for slice ID '1' and forcing memory conversion...
✅ Subset created. New shape: (144249, 7276)


In [12]:
# 1. 强制将数据矩阵 X 的数据类型从 float64 转换为 float32。
# 这通常可以将内存需求减少一半。
if adata.X.dtype != np.float32:
    adata.X = adata.X.astype(np.float32)
    print(f"-> adata.X 已转换为 {adata.X.dtype}")

# 2. 优化稀疏矩阵的索引类型 (Indices/Indptr)
# 虽然错误显示 int32，但如果它们是 int64，强制转换也能节省内存。
if sp.issparse(adata.X):
    print("-> 优化稀疏矩阵索引和指针类型...")
    # 确保是 CSR 格式
    adata.X = adata.X.tocsr(copy=False) 
    
    # 强制将索引和指针转换为 int32，以保证最小内存占用
    adata.X.indices = adata.X.indices.astype(np.int32)
    adata.X.indptr = adata.X.indptr.astype(np.int32)
    
    # 清理稀疏矩阵结构（可选，但推荐）
    adata.X.eliminate_zeros()
    adata.X.check_format()

print(f"✅ 内存优化完成。X dtype: {adata.X.dtype}, Indices dtype: {adata.X.indices.dtype}")

-> 优化稀疏矩阵索引和指针类型...
✅ 内存优化完成。X dtype: float32, Indices dtype: int32


对Cell做normalization和log transform, 以及对每个 gene 做标准化scale，让每个基因在 PCA 时贡献大致相等

In [ ]:
# 1. 基本过滤
#sc.pp.filter_cells(adata, min_genes=5)
#sc.pp.filter_genes(adata, min_cells=5)

# 2. 总计数归一化
#sc.pp.normalize_total(adata, target_sum=1e4)

# 3. log1p 变换
#sc.pp.log1p(adata)

# 4. 可选：高变基因筛选
#sc.pp.highly_variable_genes(adata, n_top_genes=2000)
#adata = adata[:, adata.var.highly_variable]
#print(f"Shape after HVG: {adata.shape}")

# 5. scale
#sc.pp.scale(adata, max_value=10)

# 6. PCA
#sc.tl.pca(adata, n_comps=50)

MemoryError: Unable to allocate 3.91 GiB for an array with shape (1049555723,) and data type int32

In [14]:
# ---------------------------------------------
# 步骤 1: 手动过滤细胞 (sc.pp.filter_cells 替代)
# ---------------------------------------------

min_genes_threshold = 5 
print(f"1. 手动计算每个细胞的非零基因数 (min_genes={min_genes_threshold})...")

# 计算每个细胞有多少个基因表达 (计数 > 0)
# .A1 用于将 Scipy 稀疏矩阵的结果（通常是 1xN 或 Nx1 矩阵）转换为 1D numpy 数组
n_genes_by_cell = np.ravel((adata.X > 0).sum(axis=1)) 

# 创建一个布尔掩码：找到要保留的细胞
cell_subset = n_genes_by_cell >= min_genes_threshold
cells_to_keep = np.sum(cell_subset)

# 将结果存储在 adata.obs 中
adata.obs['n_genes'] = n_genes_by_cell
print(f"   -> 发现 {cells_to_keep} 个细胞 ({cells_to_keep / adata.shape[0]:.2%}) 满足要求。")


# ---------------------------------------------
# 步骤 2: 手动过滤基因 (sc.pp.filter_genes 替代)
# ---------------------------------------------

min_cells_threshold = 5
print(f"2. 手动计算每个基因的表达细胞数 (min_cells={min_cells_threshold})...")

# 计算每个基因有多少个细胞表达 (计数 > 0)
n_cells_by_gene = np.ravel((adata.X > 0).sum(axis=0)) 

# 创建一个布尔掩码：找到要保留的基因
gene_subset = n_cells_by_gene >= min_cells_threshold
genes_to_keep = np.sum(gene_subset)

# 将结果存储在 adata.var 中
adata.var['n_cells'] = n_cells_by_gene
print(f"   -> 发现 {genes_to_keep} 个基因 ({genes_to_keep / adata.shape[1]:.2%}) 满足要求。")


# ---------------------------------------------
# 步骤 3: 使用切片应用子集
# ---------------------------------------------

print("\n3. 应用细胞和基因子集...")

# 使用切片操作应用过滤结果。
# AnnData 的切片操作 (adata[index, :]) 应该比内部的 .copy() 更高效。
# 注意：我们保留 .copy() 以确保得到一个新的、内存连续的对象，而不是一个 view。
# 如果这一行再次失败，请删除 .copy()
try:
    adata = adata[cell_subset, gene_subset].copy()
    print(f"✅ 过滤成功。新形状: {adata.shape}")
except MemoryError:
    # 最后的尝试：创建一个 view
    print("⚠️ .copy() 再次失败。尝试创建 view (adata = adata[...])。")
    adata = adata[cell_subset, gene_subset]
    print(f"✅ 过滤成功 (View)。新形状: {adata.shape}")


# ---------------------------------------------
# 步骤 4: 继续预处理 (例如归一化)
# ---------------------------------------------

print("\n4. 继续运行归一化...")
# 如果 adata 是一个 view (上一步删除 .copy() 的情况)，它可能不支持 inplace 操作。
# 最好在继续处理前尝试强制转换为内存对象（如果有内存剩余）。
if adata.is_view:
    adata = adata.copy() # 或者 .to_memory()

# 归一化操作通常不会有 MemoryError
sc.pp.normalize_total(adata, target_sum=1e4)
print("✅ 归一化完成。")

1. 手动计算每个细胞的非零基因数 (min_genes=5)...
   -> 发现 144249 个细胞 (100.00%) 满足要求。
2. 手动计算每个基因的表达细胞数 (min_cells=5)...
   -> 发现 7271 个基因 (99.93%) 满足要求。

3. 应用细胞和基因子集...
✅ 过滤成功。新形状: (144249, 7271)

4. 继续运行归一化...
✅ 归一化完成。


c:\Users\ouyan\anaconda3\envs\torch_env\Lib\site-packages\legacy_api_wrap\__init__.py:88: UserWarning: Some cells have zero counts
  return fn(*args_all, **kw)


In [15]:
# 1. 把每个 cell 的 PCA 特征转换成 PyTorch tensor，用作 节点特征 (node features)
X_pca = adata.obsm['X_pca']             # (N_cells, 50 PCs)
X = torch.tensor(X_pca, dtype=torch.float)

# 2. 获取每个细胞的空间坐标用来构建 空间 kNN 图
# ========== Spatial coordinates:优先使用X_spatial_coords ==========
if 'X_spatial_coords' in adata.obsm_keys():
    coords = adata.obsm['X_spatial_coords']
elif 'spatial_coords' in adata.obsm_keys():
    coords = adata.obsm['spatial_coords']
else:
    coords = adata.obs[['center_x', 'center_y']].to_numpy()

coords = coords.astype(float)
n_cells = coords.shape[0]
# 3. 获取每个细胞所属的 domain（区域、cluster），构建 domain-domain graph (DDG)
# ========== Domain labels ==========
domains = adata.obs['clust_annot'].astype('category')
domain_idx = domains.cat.codes.to_numpy()      # integer labels
n_domains = len(domains.cat.categories)

print("Nodes:", X.shape)
print("Domains:", n_domains)

Nodes: torch.Size([144249, 35])
Domains: 43


C:\Users\ouyan\AppData\Local\Temp\ipykernel_11396\3071671118.py:7: FutureWarning: Use obsm (e.g. `k in adata.obsm` or `adata.obsm.keys() | {'u'}`) instead of AnnData.obsm_keys, AnnData.obsm_keys is deprecated and will be removed in the future.
  if 'X_spatial_coords' in adata.obsm_keys():


In [17]:
# ----- Build spatial kNN graph -----
k = 8
n_cells = coords.shape[0]

nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='ball_tree')
nbrs.fit(coords)
distances, indices = nbrs.kneighbors(coords)

rows = []
cols = []
weights = []

for i in range(n_cells):
    for j, d in zip(indices[i, 1:], distances[i, 1:]):
        rows.append(i)
        cols.append(j)
        weights.append(np.exp(-d))

# Build sparse adjacency in COO format
A_spatial = sp.coo_matrix((weights, (rows, cols)), shape=(coords.shape[0], coords.shape[0]))
# Symmetrize adjacency
A_spatial = A_spatial.maximum(A_spatial.T)  # 对称化

# 🚨 修正：将矩阵转换回 COO 格式以获取 row 和 col 属性
if not isinstance(A_spatial, sp.coo_matrix):
    A_spatial = A_spatial.tocoo()
    
# Now you can safely access row, col
edge_index = torch.tensor(np.vstack([A_spatial.row, A_spatial.col]), dtype=torch.long)
edge_attr  = torch.tensor(A_spatial.data, dtype=torch.float)
print("Spatial edges:", edge_index.shape[1])

Spatial edges: 1317660


In [18]:
# Map domain names to numeric indices
domain_to_idx = {d: i for i, d in enumerate(domains.cat.categories)}
domain_per_cell = domain_idx

# Initialize domain adjacency counts
DD_counts = np.zeros((n_domains, n_domains), dtype=float)

# 假设 DD_counts 是您计算得到的域-域连接计数矩阵

# ----------------------------------------------------
# 📌 对称性检查代码
# ----------------------------------------------------

# 1. 检查 DD_counts 的维度
if DD_counts.ndim != 2 or DD_counts.shape[0] != DD_counts.shape[1]:
    print("❌ 检查失败：DD_counts 不是一个方阵，无法检查对称性。")
else:
    # 计算转置矩阵
    DD_counts_T = DD_counts.T
    
    # 检查所有元素是否相等
    # np.allclose 适用于浮点数比较，即使 DD_counts 是整数，使用它也更安全
    is_symmetric = np.allclose(DD_counts, DD_counts_T)
    
    if is_symmetric:
        print(f"✅ 检查通过：DD_counts 矩阵是对称的 (形状: {DD_counts.shape})。")
    else:
        print(f"❌ 检查失败：DD_counts 矩阵不是对称的 (形状: {DD_counts.shape})。")
        
        # 可选：检查差异最大的位置和数值
        # 计算差异矩阵的绝对值
        diff_matrix = np.abs(DD_counts - DD_counts_T)
        max_diff = np.max(diff_matrix)
        
        if max_diff > 0:
            # 找到差异最大的元素的索引
            # 注意：np.argmax 返回扁平数组的索引，需要用 np.unravel_index 转换
            max_index_flat = np.argmax(diff_matrix)
            max_index = np.unravel_index(max_index_flat, DD_counts.shape)
            
            print(f"   最大不对称差异: {max_diff:.4f}")
            print(f"   发生在索引 {max_index}: DD_counts[{max_index[0]}, {max_index[1]}] = {DD_counts[max_index]:.0f}, DD_counts_T[{max_index[0]}, {max_index[1]}] = {DD_counts_T[max_index]:.0f}")


✅ 检查通过：DD_counts 矩阵是对称的 (形状: (43, 43))。


In [19]:
# Map domain names to numeric indices
domain_to_idx = {d: i for i, d in enumerate(domains.cat.categories)}
domain_per_cell = domain_idx

# Initialize domain adjacency counts
DD_counts = np.zeros((n_domains, n_domains), dtype=float)

rows = A_spatial.row
cols = A_spatial.col

for i_cell, j_cell in zip(rows, cols):
    d_i = domain_per_cell[i_cell]
    d_j = domain_per_cell[j_cell]
    DD_counts[d_i, d_j] += 1
    DD_counts[d_j, d_i] += 1

eps = 1e-8
DD_phys = DD_counts / (DD_counts.sum(axis=1, keepdims=True) + eps)

# Compute domain mean vectors in PCA space
domain_mean = np.zeros((n_domains, X_pca.shape[1]))
domain_counts = np.zeros(n_domains)

for cell, d in enumerate(domain_per_cell):
    domain_mean[d] += X_pca[cell]
    domain_counts[d] += 1

domain_counts[domain_counts == 0] = 1
domain_mean = domain_mean / domain_counts[:, None]

from numpy.linalg import norm

DD_sem = np.zeros((n_domains, n_domains))

for i in range(n_domains):
    for j in range(n_domains):
        vi, vj = domain_mean[i], domain_mean[j]
        DD_sem[i, j] = np.dot(vi, vj) / (norm(vi)*norm(vj) + eps)

alpha = 0.5   # weight between physical and semantic
DD_combined = alpha * DD_phys + (1 - alpha) * DD_sem

# threshold weak edges
thr = 0.1
DD_mask = (DD_combined > thr).astype(float)
DD_final = DD_combined * DD_mask

DD_rows, DD_cols = np.where(DD_final > 0)
DD_weights = DD_final[DD_rows, DD_cols]

DD_edge_index = torch.tensor(np.vstack([DD_rows, DD_cols]), dtype=torch.long)
DD_edge_attr  = torch.tensor(DD_weights, dtype=torch.float)

print("DDG edges:", DD_edge_index.shape[1])

from torch_geometric.data import Data

data = Data(
    x=X,                        # PCA features
    edge_index=edge_index,      # spatial graph edges
    edge_attr=edge_attr         # spatial edge weights
)

# Add domain info + DDG
data.y_domain = torch.tensor(domain_per_cell, dtype=torch.long)
data.n_domains = n_domains
data.DD_edge_index = DD_edge_index
data.DD_edge_attr  = DD_edge_attr

print(data)

DDG edges: 503
Data(x=[144249, 35], edge_index=[2, 1317660], edge_attr=[1317660], y_domain=[144249], n_domains=43, DD_edge_index=[2, 503], DD_edge_attr=[503])


In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv  # you can switch to GCNConv or GATConv

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================
# 1. Encoder: MLP + GNN → μ, logσ
# ============================

class DRASIEncoder(nn.Module):
    def __init__(self, in_dim, hidden_mlp=128, hidden_gnn=128, latent_dim=32):
        super().__init__()
        # MLP to compress input features (PCA) before GNN
        self.mlp = nn.Sequential(
            nn.Linear(in_dim, hidden_mlp),
            nn.ReLU(),
            nn.Linear(hidden_mlp, hidden_gnn),
            nn.ReLU()
        )
        # GNN layers over spatial graph
        self.gnn1 = GraphConv(hidden_gnn, hidden_gnn)
        self.gnn2 = GraphConv(hidden_gnn, hidden_gnn)

        # Heads for VAE parameters
        self.mu_head     = nn.Linear(hidden_gnn, latent_dim)
        self.logvar_head = nn.Linear(hidden_gnn, latent_dim)

    def forward(self, x, edge_index, edge_attr):
        # x: (N, in_dim), edge_index: (2, E)
        h = self.mlp(x)
        # 核心修正：将 edge_attr 传递给 GraphConv
        h = F.relu(self.gnn1(h, edge_index, edge_attr))
        h = F.relu(self.gnn2(h, edge_index, edge_attr))
        mu     = self.mu_head(h)
        logvar = self.logvar_head(h)
        return mu, logvar


# =======================
# 2. Reparameterization
# =======================

def reparameterize(mu, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    return mu + eps * std


# =======================
# 3. Decoder: z → x̂
# =======================

class DRASIDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dec=128, out_dim=None):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden_dec),
            nn.ReLU(),
            nn.Linear(hidden_dec, out_dim)
        )

    def forward(self, z):
        return self.net(z)


# =======================
# 4. Full DRA-SI Model
# =======================

class DRASIModel(nn.Module):
    def __init__(self, in_dim, n_domains, latent_dim=32):
        super().__init__()
        self.encoder = DRASIEncoder(in_dim, latent_dim=latent_dim)
        self.decoder = DRASIDecoder(latent_dim, out_dim=in_dim)
        self.n_domains = n_domains

    def forward(self, data):
        # data.x, data.edge_index
        x = data.x
        edge_index = data.edge_index
        edge_attr = data.edge_attr

        mu, logvar = self.encoder(x, edge_index, edge_attr)
        z = reparameterize(mu, logvar)
        x_recon = self.decoder(z)

        return x_recon, mu, logvar, z


# ======================================
# 5. Loss components: Recon + KL + DDG
# ======================================

def loss_recon(x, x_recon):
    # MSE between reconstructed and original PCA features
    return F.mse_loss(x_recon, x)

def loss_kl(mu, logvar):
    # KL divergence between q(z|x) and N(0, I)
    return -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

def compute_domain_means(z, domain_idx, n_domains):
    """
    z: (N, latent_dim)
    domain_idx: (N,) int
    Returns: (n_domains, latent_dim) mean embeddings.
    """
    latent_dim = z.size(1)
    device = z.device

    sums = torch.zeros(n_domains, latent_dim, device=device)
    counts = torch.zeros(n_domains, device=device)

    # accumulate sums & counts
    sums.index_add_(0, domain_idx, z)
    ones = torch.ones_like(domain_idx, dtype=torch.float, device=device)
    counts.index_add_(0, domain_idx, ones)

    counts = counts.clamp(min=1.0)
    means = sums / counts.unsqueeze(1)
    return means

def loss_ddg(z, domain_idx, DD_edge_index, DD_edge_attr, n_domains, lambda_ddg=1.0):
    """
    z: (N, latent_dim)
    domain_idx: (N,) int
    DD_edge_index: (2, E_D)
    DD_edge_attr: (E_D,)
    """
    device = z.device
    DD_edge_index = DD_edge_index.to(device)
    DD_edge_attr  = DD_edge_attr.to(device)
    domain_idx    = domain_idx.to(device)

    domain_means = compute_domain_means(z, domain_idx, n_domains)  # (D, latent_dim)

    d_i = DD_edge_index[0]  # (E_D,)
    d_j = DD_edge_index[1]

    mu_i = domain_means[d_i]   # (E_D, latent_dim)
    mu_j = domain_means[d_j]

    diff = mu_i - mu_j
    dist_sq = (diff * diff).sum(dim=1)   # (E_D,)

    weighted = DD_edge_attr * dist_sq
    return lambda_ddg * weighted.mean()


# ======================
# 6. Total loss wrapper
# ======================

def total_loss(data, model, lambda_kl=1e-3, lambda_ddg=0.1):
    """
    data: PyG Data
    model: DRASIModel
    """
    x = data.x
    y_domain = data.y_domain
    DD_edge_index = data.DD_edge_index
    DD_edge_attr  = data.DD_edge_attr
    n_domains = data.n_domains

    x_recon, mu, logvar, z = model(data)

    L_rec = loss_recon(x, x_recon)
    L_kl  = loss_kl(mu, logvar)
    L_DDG = loss_ddg(z, y_domain, DD_edge_index, DD_edge_attr, n_domains, lambda_ddg=lambda_ddg)

    L = L_rec + lambda_kl * L_kl + L_DDG
    logs = {
        "loss": L.item(),
        "rec": L_rec.item(),
        "kl": L_kl.item(),
        "ddg": L_DDG.item()
    }
    return L, logs


Using device: cpu


In [23]:
import torch.optim as optim

in_dim = data.x.size(1)
n_domains = int(data.n_domains)

model = DRASIModel(in_dim=in_dim, n_domains=n_domains, latent_dim=32).to(device)
data = data.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

n_epochs = 100  # you can increase to 200–300 once things work

for epoch in range(1, n_epochs + 1):
    model.train()
    optimizer.zero_grad()

    L, logs = total_loss(data, model, lambda_kl=1e-3, lambda_ddg=0.1)
    L.backward()
    optimizer.step()

    if epoch % 10 == 0 or epoch == 1:
        print(
            f"Epoch {epoch:03d} | "
            f"loss={logs['loss']:.4f} | "
            f"rec={logs['rec']:.4f} | "
            f"kl={logs['kl']:.4f} | "
            f"ddg={logs['ddg']:.4f}"
        )

# After training, get latent embeddings
model.eval()
with torch.no_grad():
    _, _, _, z = model(data)   # z: (N_cells, latent_dim)

z_cpu = z.cpu().numpy()
print("Latent embedding shape:", z_cpu.shape)

# 关键步骤：将潜在表示存回 AnnData
# Z 的行数 (N_cells) 必须和 adata 的 n_obs 匹配
adata.obsm['X_drasi'] = z_cpu

print("✅ Latent embeddings successfully stored in adata.obsm['X_drasi'].")
print("New obsm keys:", adata.obsm_keys())

Epoch 001 | loss=0.0890 | rec=0.0868 | kl=0.0024 | ddg=0.0022
Epoch 010 | loss=0.0523 | rec=0.0505 | kl=0.0110 | ddg=0.0017
Epoch 020 | loss=0.0356 | rec=0.0342 | kl=0.3324 | ddg=0.0011
Epoch 030 | loss=0.0308 | rec=0.0292 | kl=1.3546 | ddg=0.0002
Epoch 040 | loss=0.0298 | rec=0.0283 | kl=1.3570 | ddg=0.0001
Epoch 050 | loss=0.0288 | rec=0.0273 | kl=1.2542 | ddg=0.0002
Epoch 060 | loss=0.0268 | rec=0.0250 | kl=1.2913 | ddg=0.0005
Epoch 070 | loss=0.0243 | rec=0.0220 | kl=1.4059 | ddg=0.0009
Epoch 080 | loss=0.0230 | rec=0.0209 | kl=1.4570 | ddg=0.0006
Epoch 090 | loss=0.0224 | rec=0.0203 | kl=1.4873 | ddg=0.0007
Epoch 100 | loss=0.0217 | rec=0.0192 | kl=1.5658 | ddg=0.0009
Latent embedding shape: (144249, 32)
✅ Latent embeddings successfully stored in adata.obsm['X_drasi'].
New obsm keys: ['X_pca', 'X_spatial_coords', 'X_umap', 'spatial_coords', 'X_drasi']


C:\Users\ouyan\AppData\Local\Temp\ipykernel_11396\203807910.py:43: FutureWarning: Use obsm (e.g. `k in adata.obsm` or `adata.obsm.keys() | {'u'}`) instead of AnnData.obsm_keys, AnnData.obsm_keys is deprecated and will be removed in the future.
  print("New obsm keys:", adata.obsm_keys())
